In [6]:
class baseRNN(nn.Module):

    def __init__(self, vocab_size, hidden_size, input_dropout_p, output_dropout_p, n_layers, rnn_cell, max_len=25):
        super(baseRNN, self).__init__()
        
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.max_len = max_len
        
        self.input_dropout_p = input_dropout_p
        self.output_dropout_p = output_dropout_p
        
        if rnn_cell.lower() == 'lstm':
            self.rnn_cell = nn.LSTM
        elif rnn_cell.lower() == 'gru':
            self.rnn_cell = nn.GRU
        else:
            raise ValueError("Unsupported RNN Cell: {0}".format(rnn_cell))

        self.input_dropout = nn.Dropout(p=input_dropout_p)

    def forward(self, *args, **kwargs):
        raise NotImplementedError()



class EncoderRNN(baseRNN):

    def __init__(self, vocab_size, embedding_size ,hidden_size= 200, input_dropout_p=0, 
                 output_dropout_p=0, n_layers=1, bidirectional=True, rnn_cell='lstm'):
        
        super(EncoderRNN, self).__init__(vocab_size, hidden_size, input_dropout_p, 
                                             output_dropout_p, n_layers, rnn_cell)

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        
        self.rnn = self.rnn_cell(embedding_size, hidden_size, n_layers,
                                 bidirectional=bidirectional, dropout=output_dropout_p,
                                 batch_first=True)

    def forward(self, words, input_lengths):
        
        batch_size = words.size()[0]
        embedded = self.embedding(words)
        embedded = self.input_dropout(embedded)
        embedded = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, batch_first= True)
        _, output = self.rnn(embedded)
        output = output[0].transpose(0,1).contiguous().view(batch_size, -1)
        return output

In [7]:
from neural_cls.util import Trainer, Loader

from neural_cls.util.utils import *

In [2]:
loader = Loader()

In [23]:
from collections import OrderedDict

In [ ]:
parameters = OrderedDict()

In [5]:
train_data, test_data , mappings = loader.load_mareview('datasets/mareview','wordvectors/glove.6B.100d.txt',100)

Found 9072 unique words (179887 in total)
Loading pretrained embeddings from wordvectors/glove.6B.100d.txt...
Found 2 unique named entity tags


In [8]:
train_data_batches = create_batches(train_data, 16, order='random')

In [9]:
datax = train_data_batches[0]

In [10]:
words = datax['words']
tags = datax['tags']

wordslen = datax['wordslen']

In [12]:
words

array([[   11,   395,    21,    11,   248,    25,    85,  1265,     7,
         1364,    40,    13,     5,    16,     7,   820,     6,  1836,
           17,   389,     4,     5,  1169,   386,    17,  2034,     6,
            5,   233,  6368,    28,   446,     4,   139,  5650,     6,
         1707,     4,    17,   765,     2],
       [ 7948,   186,    28,     3,  2133,     9,  3071,   129,   153,
            1,    96,    79,   125,  2720,     8,     3,  1798,     4,
           14,   142,   651,   221,     7,  8586,  1572,     6,  1304,
            1,  4179,     2,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [   22,    26,   318,    13,   439,    44,   379,    13,    20,
         1502,     4,    14,    73,    33,    72,   308,   748,     8,
          269,    18,    35,   153,     5,   959,   834,     7,    60,
            2,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [ 2480,  

In [15]:
tags.size()

torch.Size([32])

In [16]:
model = EncoderRNN(len(mappings['word_to_id']),100)

In [17]:
model.cuda()

EncoderRNN(
  (input_dropout): Dropout(p=0)
  (embedding): Embedding(3920, 100)
  (rnn): LSTM(100, 200, batch_first=True, bidirectional=True)
)

In [18]:
out = model(words, wordslen)

In [19]:
out.size()

torch.Size([32, 400])

In [20]:
a = torch.randn(5,3)

In [21]:
a


-0.3638  0.1268 -1.0684
 0.0629 -2.3955  0.3162
 1.7154  0.8197 -0.1374
 0.9420  0.4794  0.4138
-1.1787 -0.1383  1.0538
[torch.FloatTensor of size 5x3]

In [22]:
torch.max(a,1)

(
  0.1268
  0.3162
  1.7154
  0.9420
  1.0538
 [torch.FloatTensor of size 5], 
  1
  2
  0
  0
  2
 [torch.LongTensor of size 5])